# Here's a quick & dirty script for crawling the SEC Edgar Feed. Not meant for production.

### Setup some constants

In [85]:
# const
EDGAR_FEED_API = 'https://www.sec.gov/Archives/edgar/Feed/'
INDEX_JSON = 'index.json'
YEAR_START = 2019
COL_HREF = 'href'
COL_NAME = 'name'
KEY_DIR = 'directory'
KEY_ITEM = 'item'
# end const

### Function that will get you quarterly index uris based on a start year and end year that you give it.

In [84]:
import requests
import pandas as pd
import json

def get_qtr_index_uris(yr_st=1994, yr_end=3000):
    # get feed directorys
    data = requests.get(f'{EDGAR_FEED_API}{INDEX_JSON}').json()[KEY_DIR][KEY_ITEM]
    df = pd.read_json(json.dumps(data))
    f1 = (df[COL_NAME] >= yr_st)
    f2 = (df[COL_NAME] <= yr_end)
    df = df[f1 & f2]

    # for each year build quarterly endpoints for crawling
    qtr_apis = []
    for index, row in df.iterrows():
        url = f'{EDGAR_FEED_API}{row[COL_HREF]}{INDEX_JSON}'
        data = requests.get(url).json()[KEY_DIR][KEY_ITEM]

        qtr_df = pd.read_json(json.dumps(data))
        for qtr_index, qtr_row in qtr_df.iterrows():
            url = f'{EDGAR_FEED_API}{row[COL_HREF]}{qtr_row[COL_HREF]}{INDEX_JSON}'
            qtr_apis.append(url)
    
    return qtr_apis

qtr_index_uris = get_qtr_index_uris(YEAR_START)
print(qtr_index_uris)

['https://www.sec.gov/Archives/edgar/Feed/2019/QTR1/index.json', 'https://www.sec.gov/Archives/edgar/Feed/2019/QTR2/index.json', 'https://www.sec.gov/Archives/edgar/Feed/2019/QTR3/index.json', 'https://www.sec.gov/Archives/edgar/Feed/2019/QTR4/index.json', 'https://www.sec.gov/Archives/edgar/Feed/2020/QTR1/index.json']


### Method to process each tarball within a given quarterly index. It doesn't do anything, implementing the 'processing' is left as open-ended.

In [83]:
# test data
qtr_index_uris = [
'https://www.sec.gov/Archives/edgar/Feed/2019/QTR4/index.json',
'https://www.sec.gov/Archives/edgar/Feed/2020/QTR1/index.json'
]

def process_qtr_tarballs(qtr_uri, qtr_index_df):
    qtr_uri = f'{qtr_uri.rsplit("/", 1)[0]}/'
    for index, row in qtr_index_df.iterrows():
        tarball_uri = f'{qtr_uri}{row[COL_NAME]}'
        print(tarball_uri)
        # download the tarball, unzip in memory, throw into queueing service, whatever
               
for i in qtr_index_uris:
    data = requests.get(i).json()[KEY_DIR][KEY_ITEM]
    df = pd.read_json(json.dumps(data))
    process_qtr_tarballs(i, df)

https://www.sec.gov/Archives/edgar/Feed/2019/QTR4/20191001.nc.tar.gz
https://www.sec.gov/Archives/edgar/Feed/2019/QTR4/20191002.nc.tar.gz
https://www.sec.gov/Archives/edgar/Feed/2019/QTR4/20191003.nc.tar.gz
https://www.sec.gov/Archives/edgar/Feed/2019/QTR4/20191004.nc.tar.gz
https://www.sec.gov/Archives/edgar/Feed/2019/QTR4/20191007.nc.tar.gz
https://www.sec.gov/Archives/edgar/Feed/2019/QTR4/20191008.nc.tar.gz
https://www.sec.gov/Archives/edgar/Feed/2019/QTR4/20191009.nc.tar.gz
https://www.sec.gov/Archives/edgar/Feed/2019/QTR4/20191010.nc.tar.gz
https://www.sec.gov/Archives/edgar/Feed/2019/QTR4/20191011.nc.tar.gz
https://www.sec.gov/Archives/edgar/Feed/2019/QTR4/20191015.nc.tar.gz
https://www.sec.gov/Archives/edgar/Feed/2019/QTR4/20191016.nc.tar.gz
https://www.sec.gov/Archives/edgar/Feed/2019/QTR4/20191017.nc.tar.gz
https://www.sec.gov/Archives/edgar/Feed/2019/QTR4/20191018.nc.tar.gz
https://www.sec.gov/Archives/edgar/Feed/2019/QTR4/20191021.nc.tar.gz
https://www.sec.gov/Archives/edgar